In [1]:
import os

In [2]:
os.getcwd()

'/Users/pallavisaxena/cv2/Study/AI/NLP/FSDSBootcamp/11TextSummarization/SamsumTextSummarizer/research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'/Users/pallavisaxena/cv2/Study/AI/NLP/FSDSBootcamp/11TextSummarization/SamsumTextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import urllib
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    
    # def download_file(self):
    #     if not os.path.exists(self.config.local_data_file):
    #         req = urllib.request.Request(
    #             url=self.config.source_URL,
    #             headers={'User-Agent': 'Mozilla/5.0'}
    #         )
    #         try:
    #             with urllib.request.urlopen(req) as response, open(self.config.local_data_file, 'wb') as out_file:
    #                 data = response.read()
    #                 out_file.write(data)
    #             logger.info(f"{self.config.local_data_file} downloaded successfully!")
    #         except urllib.error.HTTPError as e:
    #             logger.error(f"HTTPError: {e.code} - {e.reason}")
    #             raise
    #         except urllib.error.URLError as e:
    #             logger.error(f"URLError: {e.reason}")
    #             raise
    #     else:
    #         logger.info(f"File already exists: {self.config.local_data_file}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-27 14:16:46,254: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-27 14:16:46,256: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 14:16:46,256: INFO: common: created directory at: artifacts]
[2024-06-27 14:16:46,257: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-27 14:16:48,463: INFO: 2114563260: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A4FE:BB339:EADCA:1133A7:667D1639
Accept-Ranges: bytes
Date: Thu, 27 Jun 2024 08:46:47 GMT
Via: 1.1 varnish
X-Served-By: cache-del21746-DEL
X-Cache: HIT